# Imports and setup

In this tutorial, we show one of the possible workflows of DL4DS. We do not aim to explore architectures, or to find the best network for the given dataset. This would require more computing power and an exhaustive exploration of the hyperparameters in DL4DS. Please read the paper and the documentation for an in-depth explanation of the library hyperparameters.

We proceed with the installation and import of DL4DS and other useful libraries.

In [ ]:
import numpy as np
import xarray as xr
import ecubevis as ecv
import dl4ds as dds
import scipy as sp
import netCDF4 as nc
import climetlab as cml

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import models

# Data

We use the downscaling benchmark dataset provided by the MAELSTROM project: https://git.ecmwf.int/projects/MLFET/repos/maelstrom-downscaling-ap5/browse/notebooks/demo_downscaling_dataset.ipynb

In [ ]:
cmlds_train = cml.load_dataset("maelstrom-downscaling-tier1", dataset="training")
cmlds_val = cml.load_dataset("maelstrom-downscaling-tier1", dataset="validation")
cmlds_test = cml.load_dataset("maelstrom-downscaling-tier1", dataset="testing")

We convert the downloaded objects to xarray datasets. You may need to execute the following cell a second time if you get an OSError (``OSError: [Errno -51] NetCDF: Unknown file format``).

In [ ]:
t2m_hr_train = cmlds_train.to_xarray().t2m_tar
t2m_hr_test = cmlds_test.to_xarray().t2m_tar
t2m_hr_val = cmlds_val.to_xarray().t2m_tar

t2m_lr_train = cmlds_train.to_xarray().t2m_in
t2m_lr_test = cmlds_test.to_xarray().t2m_in
t2m_lr_val = cmlds_val.to_xarray().t2m_in

z_hr_train = cmlds_train.to_xarray().z_tar
z_hr_test = cmlds_test.to_xarray().z_tar
z_hr_val = cmlds_val.to_xarray().z_tar

z_lr_train = cmlds_train.to_xarray().z_in
z_lr_test = cmlds_test.to_xarray().z_in
z_lr_val = cmlds_val.to_xarray().z_in

In [ ]:
z_hr_train

Now let's visualize the xr.Dataset for the t2m variable with ECUBEVIS (https://github.com/carlos-gg/ecubevis.git).

In [ ]:
ecv.plot(t2m_hr_train)

Here we take care of the scaling/normalization of values before training our networks. In this exmample, we center wrt the global mean and scale wrt the global standard deviation. 

In [ ]:
t2m_scaler_train = dds.StandardScaler(axis=None)
t2m_scaler_train.fit(t2m_hr_train)  
y_train = t2m_scaler_train.transform(t2m_hr_train)
y_test = t2m_scaler_train.transform(t2m_hr_test)
y_val = t2m_scaler_train.transform(t2m_hr_val)

x_train = t2m_scaler_train.transform(t2m_lr_train)
x_test = t2m_scaler_train.transform(t2m_lr_test)
x_val = t2m_scaler_train.transform(t2m_lr_val)

z_scaler_train = dds.StandardScaler(axis=None)
z_scaler_train.fit(z_hr_train)  
y_z_train = z_scaler_train.transform(z_hr_train)
y_z_test = z_scaler_train.transform(z_hr_test)
y_z_val = z_scaler_train.transform(z_hr_val)

x_z_train = z_scaler_train.transform(z_lr_train)
x_z_test = z_scaler_train.transform(z_lr_test)
x_z_val = z_scaler_train.transform(z_lr_val)

In [ ]:
ecv.plot((x_train[0], y_train[0]), subplot_titles=('t2m coarsened', 't2m highres'))

In [ ]:
ecv.plot((x_z_train[0], y_z_train[0]), subplot_titles=('geopotential coarsened', 'geopotential highres'))

We add a last unitary dimension which we call ``channel``.

In [ ]:
y_train = y_train.expand_dims(dim='channel', axis=-1)
y_test = y_test.expand_dims(dim='channel', axis=-1)
y_val = y_val.expand_dims(dim='channel', axis=-1)

x_train = x_train.expand_dims(dim='channel', axis=-1)
x_test = x_test.expand_dims(dim='channel', axis=-1)
x_val = x_val.expand_dims(dim='channel', axis=-1)

y_z_train = y_z_train.expand_dims(dim='channel', axis=-1)
y_z_test = y_z_test.expand_dims(dim='channel', axis=-1)
y_z_val = y_z_val.expand_dims(dim='channel', axis=-1)

x_z_train = x_z_train.expand_dims(dim='channel', axis=-1)
x_z_test = x_z_test.expand_dims(dim='channel', axis=-1)
x_z_val = x_z_val.expand_dims(dim='channel', axis=-1)

These are the resulting shapes of the input arrays:

In [ ]:
print(y_train.shape, y_test.shape, y_val.shape)
print(x_train.shape, x_test.shape, x_val.shape)

print(x_z_train.shape, x_z_test.shape, x_z_val.shape)
print(y_z_train.shape, y_z_test.shape, y_z_val.shape)

# Training

For training, DL4DS takes:

* HR data _[mandatory, highres grid]_, 
* LR data _[optional, coarse grid]_, 
* static variables _[optional, highres grid]_,
* time-varyibng variables _[optional, coarse to highres grid]_. 

In this example, we do not use the LR data because it is just a coarsened version (via interpolation) of the HR t2m and z data. DL4DS carries out this interpolation on the fly by using the helping function ``dds.create_pair_hr_lr()`` (not to be called by the user). This process is done automatically inside the training loop (by calling one of the two Trainer classes in DL4DS), which we examplify here with the ``spc`` upsampling.

In [ ]:
_ = dds.create_pair_hr_lr(y_train.values[0], None, 'spc', 8, None, None, y_z_train.values[0], None, True, interpolation='inter_area')

Below is an example of a call to the SupervisedTrainer class to run the training loop for 100 epochs. In this case, we train a network with a ResNet backbone, 8 residual blocks, in post-upsampling via subpixel convolution (implicit training pairs), and a localized convolutional block. 

In [ ]:
ARCH_PARAMS = dict(n_filters=8,
                   n_blocks=8,
                   normalization=None,
                   dropout_rate=0.0,
                   dropout_variant='spatial',
                   attention=False,
                   activation='relu',
                   localcon_layer=True)

trainer = dds.SupervisedTrainer(
    backbone='resnet',
    upsampling='spc', 
    data_train=y_train, 
    data_val=y_val,
    data_test=y_test,
    data_train_lr=None, # here you can pass the LR dataset for training with explicit paired samples
    data_val_lr=None, # here you can pass the LR dataset for training with explicit paired samples
    data_test_lr=None, # here you can pass the LR dataset for training with explicit paired samples
    scale=8,
    time_window=None, 
    static_vars=None,
    predictors_train=[y_z_train],
    predictors_val=[y_z_val],
    predictors_test=[y_z_test],
    interpolation='inter_area',
    patch_size=None, 
    batch_size=60, 
    loss='mae',
    epochs=100, 
    steps_per_epoch=None, 
    validation_steps=None, 
    test_steps=None, 
    learning_rate=(1e-3, 1e-4), lr_decay_after=1e4,
    early_stopping=False, patience=6, min_delta=0, 
    save=False, 
    save_path=None,
    show_plot=True, verbose=True, 
    device='GPU', 
    **ARCH_PARAMS)

trainer.run()

# Inference

Let's evaluate the results on holdout data -- the test split of the benchmark dataset that has not been used while training (updating the network weights).

In [ ]:
pred = dds.Predictor(
    trainer, 
    y_test, 
    scale=8, 
    array_in_hr=True,
    static_vars=None, 
    predictors=[y_z_test], 
    time_window=None,
    interpolation='inter_area', 
    batch_size=8,
    scaler=t2m_scaler_train,
    save_path=None,
    save_fname=None,
    return_lr=True,
    device='CPU')

unscaled_y_pred, coarsened_array = pred.run()

Below is the coarsened version of the holdout, passed to the trained model for inference:

In [ ]:
ind = 100
ecv.plot((coarsened_array[ind][:,:,0], coarsened_array[ind][:,:,1]))

In [ ]:
unscaled_y_test = t2m_scaler_train.inverse_transform(y_test)

Finally, let's see compare the groundtruth HR t2m and the downscaled t2m obtained with DL4DS for a single time step. 

In [ ]:
ecv.plot((unscaled_y_test[ind].values, unscaled_y_pred[ind]), subplot_titles=('groundtruth t2m', 'downscaled t2m'))

Please take into account that we are using a dummy dataset with few samples, and are training in supervised fashion a single possible DL4DS architecture, without tuning hyperparameters. In order to find the best model, you should compute relevant metrics for your problem. This is out of the scope of this tutorial.